In [105]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [106]:
train_raw_data=pd.read_csv('train.csv')
test_raw_data=pd.read_csv('test.csv')

In [107]:
trainrow=train_raw_data.shape[0]
testrow=test_raw_data.shape[0]
y_train=train_raw_data['Survived'].copy()
train_raw_data=train_raw_data.drop(['Survived'],1)

In [108]:
combine=pd.concat([train_raw_data,test_raw_data])
combine.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [109]:
print(combine.shape)
print(train_raw_data.shape)
print(test_raw_data.shape)

(1309, 11)
(891, 11)
(418, 11)


In [110]:
combine.isnull().sum()

PassengerId       0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [111]:
#here we can se only three features has missing data
#Emabarked lets replace it with most frequent data
combine['Embarked']=combine['Embarked'].fillna(combine['Embarked'].value_counts()[0])

In [112]:
#here we can see cabin does not matter to Survive so lets delete it
combine=combine.drop(['Cabin'],1)

In [113]:
combine.loc[combine['Fare'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,S


In [114]:
#fare depend on Pclass,Sex, and age
combine['Fare']=combine['Fare'].fillna(combine.loc[(combine['Pclass']==3) & (combine['Sex']=="male") & (combine['Age']<65) & (combine['Age']>55)].dropna()['Fare'].mean())

In [115]:
combine.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [116]:
passengerids=test_raw_data['PassengerId']
combine=combine.drop(['PassengerId','Ticket'],1)
passengerids

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [117]:
combine.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S


In [118]:
combine['familysize']=combine['SibSp']+combine['Parch']+1
combine.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,familysize
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,2
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,2
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,1
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,2
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,1


In [119]:
combine['Title'] = combine.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
combine.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,familysize,Title
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,2,Mr
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,2,Mrs
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,1,Miss
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,2,Mrs
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,1,Mr


In [120]:
combine.Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'Countess',
       'Jonkheer', 'Dona'], dtype=object)

In [121]:
combine['Title'].value_counts()

Mr          757
Miss        260
Mrs         197
Master       61
Dr            8
Rev           8
Col           4
Ms            2
Major         2
Mlle          2
Jonkheer      1
Mme           1
Dona          1
Sir           1
Countess      1
Lady          1
Don           1
Capt          1
Name: Title, dtype: int64

In [122]:
combine['Title']=combine['Title'].map({"Mr":0,"Miss":1,"Mrs":2,"Master":3,"Dr":3,"Rev":3,"Major":3,"Col":3,"Mlle":3,"Don":3,"Jonkheer":3,"Countess":3,"Sir":0,"Capt":3,"Mme":2,"Lady":1,"Ms":0,"Dona":3})
combine.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,familysize,Title
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,2,0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,2,2
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,1,1
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,2,2
4,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,1,0


In [123]:
combine=combine.drop(['Name'],1)
combine.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,familysize,Title
0,3,male,22.0,1,0,7.2500,S,2,0
1,1,female,38.0,1,0,71.2833,C,2,2
2,3,female,26.0,0,0,7.9250,S,1,1
3,1,female,35.0,1,0,53.1000,S,2,2
4,3,male,35.0,0,0,8.0500,S,1,0


In [124]:
combine=combine.drop(['SibSp','Parch'],1)
combine.head()

,Pclass,Sex,Age,Fare,Embarked,familysize,Title
0,3,male,22.0,7.2500,S,2,0
1,1,female,38.0,71.2833,C,2,2
2,3,female,26.0,7.9250,S,1,1
3,1,female,35.0,53.1000,S,2,2
4,3,male,35.0,8.0500,S,1,0


In [125]:
combine['Sex']=combine['Sex'].map({'male':0,'female':1})
combine.head()

,Pclass,Sex,Age,Fare,Embarked,familysize,Title
0,3,0,22.0,7.2500,S,2,0
1,1,1,38.0,71.2833,C,2,2
2,3,1,26.0,7.9250,S,1,1
3,1,1,35.0,53.1000,S,2,2
4,3,0,35.0,8.0500,S,1,0


In [126]:
combine.Pclass.unique()

array([3, 1, 2], dtype=int64)

In [127]:
for i in range(0,2):
    for j in range(0,3):
        temp_dataset=combine[(combine['Sex']==i) &  (combine['Pclass']==j+1)]['Age'].dropna()
        temp_dataset
        #print(str(temp_dataset.median())+"  "+str(i)+"  "+str(j+1))
        combine.loc[(combine.Age.isnull()) & (combine.Sex==i) & (combine.Pclass==j+1),'Age']=int(temp_dataset.median())

In [128]:
combine.isnull().sum()

Pclass        0
Sex           0
Age           0
Fare          0
Embarked      0
familysize    0
Title         0
dtype: int64

In [129]:
combine.head()

,Pclass,Sex,Age,Fare,Embarked,familysize,Title
0,3,0,22.0,7.2500,S,2,0
1,1,1,38.0,71.2833,C,2,2
2,3,1,26.0,7.9250,S,1,1
3,1,1,35.0,53.1000,S,2,2
4,3,0,35.0,8.0500,S,1,0


In [130]:
combine['Embarked'].unique()

array(['S', 'C', 'Q', 914], dtype=object)

In [132]:
combine['Embarked']=combine['Embarked'].map({'S':1,'C':2,'Q':3,914:4})

In [133]:
combine.head()

,Pclass,Sex,Age,Fare,Embarked,familysize,Title
0,3,0,22.0,7.2500,1,2,0
1,1,1,38.0,71.2833,2,2,2
2,3,1,26.0,7.9250,1,1,1
3,1,1,35.0,53.1000,1,2,2
4,3,0,35.0,8.0500,1,1,0


In [134]:
x_train=combine.iloc[:trainrow]
x_test=combine.iloc[trainrow:]

In [136]:
combine=pd.get_dummies(columns=['Pclass','Sex','Embarked','Title'],data=combine)
combine.head()

,Age,Fare,familysize,Pclass_1,Pclass_2,Pclass_3,Sex_0,Sex_1,Embarked_1,Embarked_2,Embarked_3,Embarked_4,Title_0,Title_1,Title_2,Title_3
0,22.0,7.2500,2,0,0,1,1,0,1,0,0,0,1,0,0,0
1,38.0,71.2833,2,1,0,0,0,1,0,1,0,0,0,0,1,0
2,26.0,7.9250,1,0,0,1,0,1,1,0,0,0,0,1,0,0
3,35.0,53.1000,2,1,0,0,0,1,1,0,0,0,0,0,1,0
4,35.0,8.0500,1,0,0,1,1,0,1,0,0,0,1,0,0,0


In [137]:
from sklearn.preprocessing import StandardScaler

In [138]:
scaler=StandardScaler()
scaler.fit(x_train)
x_scaled_train=scaler.transform(x_train)
x_scaled_train

array([[ 0.82737724, -0.73769513, -0.53885438, ..., -0.56876427,
         0.05915988, -0.73137042],
       [-1.56610693,  1.35557354,  0.66141187, ...,  0.97626307,
         0.05915988,  1.35725473],
       [ 0.82737724,  1.35557354, -0.23878782, ..., -0.56876427,
        -0.56097483,  0.31294215],
       ...,
       [ 0.82737724,  1.35557354, -0.53885438, ..., -0.56876427,
         1.29942929,  0.31294215],
       [-1.56610693, -0.73769513, -0.23878782, ...,  0.97626307,
        -0.56097483, -0.73137042],
       [ 0.82737724, -0.73769513,  0.21131203, ...,  2.52129041,
        -0.56097483, -0.73137042]])

In [140]:
x_scaled_test=scaler.transform(x_test)
x_scaled_test

In [143]:
from sklearn.linear_model import LogisticRegression
reg=LogisticRegression()
reg.fit(x_scaled_train,y_train)
print(reg.score(x_scaled_train,y_train))
y_pred=reg.predict(x_scaled_test)
y_pred

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [152]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(x_scaled_train,y_train,early_stopping_rounds=5, 
             eval_set=[(x_scaled_train, y_train)], 
             verbose=False)
print(xgb.score(x_scaled_train,y_train))
y_pred=xgb.predict(x_scaled_test)
y_pred

0.9057239057239057


array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [153]:
passengerids.head()

0    892
1    893
2    894
3    895
4    896
Name: PassengerId, dtype: int64

In [154]:
submission = pd.DataFrame({
        "PassengerId": passengerids,
        "Survived": y_pred
    })
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [155]:
submission.to_csv('submission.csv', index=False)